## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/tdfXFY2y), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.  

In [1]:
import chassisml
import pickle
import cv2
import torch
import getpass
import numpy as np
import torchvision.models as models
from torchvision import transforms

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Prepare model, labels, and data transform
Initialize anything here that should persist across inference runs

In [3]:
model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)
model.eval()

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])       

device = 'cpu'

def preprocess(input_bytes):
    decoded = cv2.imdecode(np.frombuffer(input_bytes, np.uint8), -1)
    img_t = transform(decoded)
    batch_t = torch.unsqueeze(img_t, 0).to(device)
    return batch_t

def postprocess(num_detections, predictions):
    inference_result = {
        "detections": [
            {
                "xMin": predictions["boxes"][i].detach().cpu().numpy().tolist()[0],
                "xMax": predictions["boxes"][i].detach().cpu().numpy().tolist()[2],
                "yMin": predictions["boxes"][i].detach().cpu().numpy().tolist()[1],
                "yMax": predictions["boxes"][i].detach().cpu().numpy().tolist()[3],
                "class": labels[predictions["labels"][i].detach().cpu().item()],
                "classProbability": predictions["scores"][i].detach().cpu().item(),
            } for i in range(num_detections)
        ]
    }

    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }    
    return structured_output

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [5]:
def process(input_bytes):
    
    # preprocess
    batch_t = preprocess(input_bytes)
    
    # run inference
    predictions = model(batch_t)[0]
    num_detections = len(predictions["boxes"])
    
    # postprocess
    structured_output = postprocess(num_detections, predictions)
    
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [6]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `process_fn` defined above

In [ ]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/airplane.jpg'
results = chassis_model.test(sample_filepath)
print(results)

In [8]:
# test environment and model within Chassis service, must pass filepath here:
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

Starting test job... Ok!
{'model_output': 'Single input prediction:\n\nb\'{"data":{"result":{"detections":[{"xMin":112.44477844238281,"xMax":594.023681640625,"yMin":10.943532943725586,"yMax":201.69760131835938,"class":"airplane","classProbability":0.9996569156646729}]},"explanation":null,"drift":null}}\'\n'}


## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [ ]:
MODZY_URL = "https://demo.modzy.engineering/api"

response = chassis_model.publish(
    model_name="PyTorch Faster R-CNN Object Detection",
    model_version="0.0.2",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    modzy_sample_input_path=sample_filepath,
    modzy_api_key=modzy_api_key,
    modzy_url=MODZY_URL
)

In [13]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

In [14]:
if final_status["result"] is not None:
    print("New model URL: {}".format(final_status["result"]["container_url"]))
else:
    print("Chassis job failed \n\n {}".format(final_status))

New model URL: https://demo.modzy.engineering/models/ddghb2gieb/0.0.2


## Run sample job
Submit inference job to our newly-deploy model running on Modzy

In [15]:
from modzy import ApiClient

client = ApiClient(base_url=MODZY_URL, api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)

ApiObject({
  "data": {
    "drift": null,
    "explanation": null,
    "result": {
      "detections": [
        {
          "class": "airplane",
          "classProbability": 0.9996569156646729,
          "xMax": 594.023681640625,
          "xMin": 112.44480895996094,
          "yMax": 201.69760131835938,
          "yMin": 10.943532943725586
        }
      ]
    }
  }
})
